In [1]:
# pip install qwen-vl-utils[decord]

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

# default: Load the model on the available device(s)
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
#             },
#             {"type": "text", "text": "Describe this image."},
#         ],
#     }
# ]

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/home/user/Downloads/_public/4_t_4.jpg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

["The image depicts a serene beach scene with a woman and a dog. The woman is sitting on the sand, wearing a plaid shirt and black pants, and appears to be smiling. She is holding the dog's paw in a high-five gesture. The dog, which is a large breed, is wearing a harness and is sitting on the sand, facing the woman. The background shows the ocean with gentle waves, and the sky is clear with a soft light, suggesting either sunrise or sunset. The overall atmosphere is peaceful and joyful."]


In [11]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/home/user/Downloads/_public/4_t_4.jpg",
            },
            {"type": "text", "text": "распознай рукописный текст и верни только его"},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=456)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The handwritten text in the image appears to be a record of a reward or recognition. Here is the transcription of the text:\n\n```\nСведения о\nНаграждениях\nТК № 3575237\n\nНаименование, \nдата и номер документа, \nна основании которого \nвнесена запись\n\nОбщество охранения \nответственности \nВеселая карусель-77 \n(1000, ВК - 444)\n\n03 13 10 2017 \nНагражден почётной \nграмотой за добросовестный \nтруд\n```\n\nThe transcription includes:\n\n- "Сведения о" (Information about)\n- "Награждениях" (Recognition)\n- "ТК № 3575237" (TK No 3575237)\n- "Наименование, дата и номер документа, на основании которого внесена запись" (Name, date, and number of the document, on the basis of which the entry was made)\n- "Общество охранения ответственности" (Organization for the protection of responsibility)\n- "Веселая карусель-77" (Happy Carousel-77)\n- "(1000, ВК - 444)" (1000, VK - 444)\n- "03 13 10 2017" (03 October 2017)\n- "Награжден почётной грамотой за добросовестный труд" (Awarded a honor

In [9]:
# pip install 'accelerate>=0.26.0'